<a href="https://colab.research.google.com/github/NegociandoSemMedo/Avali-o_On-line_PI_CP-CTI_PEN/blob/main/Avalia%C3%A7%C3%A3o_On_line_PI_CP_CTI_PEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Avalição On-line PI_CP-CTI_PEN
Vou usar o Colab Notbook para o script no python.

In [ ]:
# Preparação de ambiente e importações de bibliotecas

# Importa a biblioteca pandas para manipulação de dados
import pandas as pd

# Importa as bibliotecas do google.colab.files para permitir o upload de arquivos
from google.colab import files


In [5]:
# Exibe uma mensagem para o usuário
print("Por favor, selecione o arquivo 'arq_dengue.csv'.")

# Solicita que o usuário faça o upload de um arquivo
uploaded = files.upload()

# Verifica se algum arquivo foi carregado
if uploaded:
    # O nome do arquivo será a primeira (e geralmente única) chave no dicionário uploaded
    file_name = next(iter(uploaded))
    print(f"Arquivo '{file_name}' carregado com sucesso.")

    # Lê o arquivo CSV para um DataFrame pandas
    # Usamos 'sep=;' para indicar que o delimitador de colunas é o ponto e vírgula
    # Isso é crucial, pois seu arquivo usa ';' e não a vírgula padrão.
    df_dengue = pd.read_csv(file_name, sep=';')

    # Exibe as primeiras 5 linhas do DataFrame para verificar o carregamento
    # 'df_dengue.head()' mostra um resumo dos dados, útil para inspeção inicial
    print("\nPrimeiras 5 linhas do DataFrame 'df_dengue' (após carregamento com separador ';'):")
    print(df_dengue.head())

    # Exibe informações sobre as colunas, como tipos de dados e valores não nulos
    # Isso nos ajuda a identificar problemas de carregamento, como colunas com espaços extras.
    print("\nInformações do DataFrame:")
    df_dengue.info()

    # Exibe os nomes das colunas para identificar possíveis espaços em branco ou caracteres indesejados
    print("\nNomes das colunas (para verificação de espaços/caracteres):")
    print(df_dengue.columns.tolist())

else:
    print("Nenhum arquivo foi selecionado ou carregado.")

Por favor, selecione o arquivo 'arq_dengue.csv'.


Saving arq_dengue.csv to arq_dengue (2).csv
Arquivo 'arq_dengue (2).csv' carregado com sucesso.

Primeiras 5 linhas do DataFrame 'df_dengue' (após carregamento com separador ';'):
   paciente  febre  dor_muscular falta_apetite manchas_vermelhas dengue
0         1    38.6          sim           sim               sim    sim
1         2    36.4          nao           nao               nao    nao
2         3    37.5          sim           sim               nao    sim
3         4    38.9          sim           sim               sim    sim
4         5    39.3          sim           sim               sim    sim

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   paciente           20 non-null     int64  
 1   febre              20 non-null     float64
 2   dor_muscular       20 non-null     object 
 3   falta_apetite 

In [7]:
# Verifica se o DataFrame 'df_dengue' existe e não está vazio
if 'df_dengue' in locals() and not df_dengue.empty:
    print("\nIniciando a limpeza dos nomes das colunas e a transformação dos dados 'sim'/'nao' para binários (1/0)...")

    # 1. Limpar os nomes das colunas (remover espaços em branco extras)
    # df_dengue.columns.str.strip() remove espaços em branco do início e fim de cada nome de coluna
    df_dengue.columns = df_dengue.columns.str.strip()
    print("\nNomes das colunas após a limpeza:")
    print(df_dengue.columns.tolist())

    # 2. Definir as colunas que precisam ser transformadas de 'sim'/'nao' para 1/0
    # Com base no seu exemplo, as colunas são: dor_muscular, falta_apetite, manchas_vermelhas, dengue
    colunas_para_transformar = ['dor_muscular', 'falta_apetite', 'manchas_vermelhas', 'dengue']

    # Itera sobre cada coluna que definimos para transformação
    for column in colunas_para_transformar:
        # Verifica se a coluna realmente existe no DataFrame após a limpeza dos nomes
        if column in df_dengue.columns:
            print(f"  Processando coluna: '{column}'")

            # Converte a coluna para o tipo string e para minúsculas para padronizar
            # Isso garante que 'Sim', 'SIM', 'Nao', 'NAO' etc. sejam tratados corretamente
            temp_series = df_dengue[column].astype(str).str.lower()

            # Mapeia 'sim' para 1 e 'nao' para 0.
            # Valores que não são 'sim' nem 'nao' se tornarão NaN após o map.
            mapeamento = {'sim': 1, 'nao': 0}
            df_dengue.loc[:, column] = temp_series.map(mapeamento)

            # Lida com valores NaN resultantes de valores que não eram 'sim' nem 'nao'.
            # Para este problema, vamos assumir que qualquer coisa que não seja 'sim'
            # deve ser tratada como 'nao' (0). Se houver outros valores que não fazem
            # sentido, eles serão preenchidos com 0.
            df_dengue.loc[:, column].fillna(0, inplace=True)

            # Converte a coluna para tipo inteiro para representar os binários (1 e 0)
            df_dengue.loc[:, column] = df_dengue[column].astype(int)
        else:
            print(f"  AVISO: Coluna '{column}' não encontrada no DataFrame e não será transformada. Verifique se o nome está correto.")

    # Exibe as primeiras 5 linhas do DataFrame atualizado para verificar a transformação
    print("\nRegistros atualizados após a transformação 'sim'/'nao' para binários:")
    print(df_dengue.head())

    # Exibe os tipos de dados para confirmar a conversão para numérico (int)
    print("\nTipos de dados após a transformação:")
    print(df_dengue.dtypes)

    # Verificação adicional para garantir que a coluna 'dengue' foi transformada
    if 'dengue' in df_dengue.columns and df_dengue['dengue'].dtype == 'int64':
        print(f"\nA coluna 'dengue' e outras colunas de sintomas foram transformadas com sucesso para binários (1/0).")
    else:
        print(f"\nATENÇÃO: A coluna 'dengue' pode não ter sido transformada como esperado. Verifique os dados.")

else:
    print("O DataFrame 'df_dengue' não foi carregado ou está vazio. Por favor, execute o Passo 1 primeiro.")


Iniciando a limpeza dos nomes das colunas e a transformação dos dados 'sim'/'nao' para binários (1/0)...

Nomes das colunas após a limpeza:
['paciente', 'febre', 'dor_muscular', 'falta_apetite', 'manchas_vermelhas', 'dengue']
  Processando coluna: 'dor_muscular'
  Processando coluna: 'falta_apetite'
  Processando coluna: 'manchas_vermelhas'
  Processando coluna: 'dengue'

Registros atualizados após a transformação 'sim'/'nao' para binários:
   paciente  febre dor_muscular falta_apetite manchas_vermelhas dengue
0         1   38.6            1             1                 1      1
1         2   36.4            0             0                 0      0
2         3   37.5            1             1                 0      1
3         4   38.9            1             1                 1      1
4         5   39.3            1             1                 1      1

Tipos de dados após a transformação:
paciente               int64
febre                float64
dor_muscular          object
fal

<ipython-input-7-2f7065aa7a27>:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dengue.loc[:, column].fillna(0, inplace=True)
<ipython-input-7-2f7065aa7a27>:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [9]:
import pandas as pd # Garante que pandas está importado
from sklearn.model_selection import train_test_split # Para dividir os dados
from sklearn.neighbors import KNeighborsClassifier # O algoritmo KNN
from sklearn.metrics import confusion_matrix, accuracy_score # Para avaliar o modelo

# Verifica se o DataFrame 'df_dengue' existe e não está vazio
if 'df_dengue' in locals() and not df_dengue.empty:
    print("\nIniciando a etapa de Mineração de Dados (Revisão para Tipo de Alvo)...")

    # 1. Preparação dos dados para o modelo
    # 'X' (features/características): Todas as colunas, exceto 'paciente' e 'dengue'.
    X = df_dengue.drop(['paciente', 'dengue'], axis=1)

    # 'y' (target/alvo): A coluna 'dengue'.
    # É CRUCIAL GARANTIR QUE 'y' SEJA TRATADO COMO CLASSE DISCRETA (CATEGÓRICA).
    # Vamos converter explicitamente para 'category' e depois para 'int' (se já não for).
    # A biblioteca scikit-learn geralmente lida bem com inteiros como classes,
    # mas a mensagem de erro sugere que algo está misturando tipos.
    y = df_dengue['dengue']

    # --- VERIFICAÇÃO E CONVERSÃO EXPLÍCITA PARA TIPO DISCRETO ---
    # Primeiro, garanta que seja int. Se ainda houver problemas, tente converter para category.
    if y.dtype != 'int64':
        try:
            y = y.astype(int)
            print(f"Alvo 'y' convertido para tipo inteiro: {y.dtype}")
        except ValueError:
            print("AVISO: Não foi possível converter o alvo 'y' diretamente para int. Verifique os valores.")
            # Se a conversão para int falhar, pode ser que existam outros valores além de 0 e 1.
            # Nesse caso, vamos tentar forçar como tipo 'category'
            y = y.astype('category').cat.codes
            print(f"Alvo 'y' convertido para tipo categórico codificado: {y.dtype}")
            # Esta linha acima irá mapear categorias únicas para 0, 1, 2...
            # Se seus dados são estritamente 'sim'/'nao' que viraram 1/0, então cat.codes manterá 1/0.
    # -----------------------------------------------------------

    print(f"\nCaracterísticas (X) shape: {X.shape}")
    print(f"Alvo (y) shape: {y.shape}")
    print("\nPrimeiras linhas de X (Características):")
    print(X.head())
    print("\nPrimeiras linhas de y (Alvo - 'dengue'):")
    print(y.head())
    print(f"Tipo de dado final do alvo (y): {y.dtype}") # Para depuração

    # 2. Divisão da base de dados em treinamento e teste
    # random_state=42 para reprodutibilidade.
    X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.30, random_state=42)

    print(f"\nTamanho do conjunto de treinamento (X_treinamento): {X_treinamento.shape}")
    print(f"Tamanho do conjunto de teste (X_teste): {X_teste.shape}")

    # 3. Selecionar o algoritmo KNeighborsClassifier e treinar o modelo
    modelo_knn = KNeighborsClassifier(n_neighbors=3)

    # .fit() treina o modelo usando os dados de treinamento (X_treinamento e y_treinamento).
    print("\nTreinando o modelo KNeighborsClassifier...")
    modelo_knn.fit(X_treinamento, y_treinamento) # A linha que estava dando erro
    print("Modelo treinado com sucesso!")

    # 4. Aplicar o método de predição
    print("\nRealizando predições no conjunto de teste...")
    y_predicoes = modelo_knn.predict(X_teste)
    print("Previsões realizadas!")

    # Exibe algumas das previsões
    print("\nAlgumas das previsões do modelo (primeiras 10):")
    print(y_predicoes[:10])
    print("\nValores reais correspondentes (primeiras 10):")
    print(y_teste.head(10).tolist())

    # 5. Apresentar a matriz de confusão e a acurácia do modelo
    print("\n--- Avaliação do Modelo ---")

    matriz_confusao = confusion_matrix(y_teste, y_predicoes)
    print("\nMatriz de Confusão:")
    print(matriz_confusao)

    acuracia = accuracy_score(y_teste, y_predicoes)
    print(f"\nAcurácia do Modelo: {acuracia:.4f}")

else:
    print("O DataFrame 'df_dengue' não foi carregado ou está vazio. Por favor, execute os passos anteriores primeiro.")


Iniciando a etapa de Mineração de Dados (Revisão para Tipo de Alvo)...
Alvo 'y' convertido para tipo inteiro: int64

Características (X) shape: (20, 4)
Alvo (y) shape: (20,)

Primeiras linhas de X (Características):
   febre dor_muscular falta_apetite manchas_vermelhas
0   38.6            1             1                 1
1   36.4            0             0                 0
2   37.5            1             1                 0
3   38.9            1             1                 1
4   39.3            1             1                 1

Primeiras linhas de y (Alvo - 'dengue'):
0    1
1    0
2    1
3    1
4    1
Name: dengue, dtype: int64
Tipo de dado final do alvo (y): int64

Tamanho do conjunto de treinamento (X_treinamento): (14, 4)
Tamanho do conjunto de teste (X_teste): (6, 4)

Treinando o modelo KNeighborsClassifier...
Modelo treinado com sucesso!

Realizando predições no conjunto de teste...
Previsões realizadas!

Algumas das previsões do modelo (primeiras 10):
[1 0 1 0 1 1]

Valo